In [3]:
!pip install -r requirements.txt
import numpy as np

Defaulting to user installation because normal site-packages is not writeable


In [38]:
from PIL import Image

def save_images_as_png(images_array, prefix="image"):
    for i, image_array in enumerate(images_array):
        # Convert the image array to PIL Image
        image = Image.fromarray(image_array)
        # Save the image as PNG with a sequential filename
        filename = f"{prefix}_{i}.png"
        image.save(f"images/{filename}")

def read_images_from_binary_file(path):
    with open(path,"rb") as file:
        magic_number=file.read(4)
        number_of_items = int.from_bytes(file.read(4), byteorder='big')
        number_of_rows = int.from_bytes(file.read(4), byteorder='big')
        number_of_columns = int.from_bytes(file.read(4), byteorder='big')
        image_data=file.read()
        size=number_of_rows*number_of_columns

    images_array=np.frombuffer(image_data,dtype=np.uint8)
    images_array = images_array.reshape((number_of_items, number_of_columns, number_of_rows))
    return images_array

def read_training_set_label_file(filename):
    with open(filename, 'rb') as file:
        # Read the magic number and other metadata
        magic_number = int.from_bytes(file.read(4), byteorder='big')
        num_labels = int.from_bytes(file.read(4), byteorder='big')

        # Read the label data
        label_data = np.frombuffer(file.read(num_labels), dtype=np.uint8)

    return label_data

training_data=read_images_from_binary_file("data/train-images.idx3-ubyte")
training_label=read_training_set_label_file("data/train-labels.idx1-ubyte")
testing_data=read_images_from_binary_file("data/t10k-images.idx3-ubyte")
testing_label=read_training_set_label_file("data/train-labels.idx1-ubyte")



In [96]:
learning_rate=1
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sigmoid_x = sigmoid(x)
    return sigmoid_x * (1 - sigmoid_x)

class Layer:
    def __init__(self, Values):
      self.values = Values
      self.zvalues= None
      self.weights = None
      self.biases = None
      self.delta_weights= None
      self.delta_biases = None
      self.next_layer=None
      self.previous_layer=None
      
    def nextLayer(self,nodes):
      if nodes!=0:
         self.weights = np.random.rand(self.values.size, nodes) * 0.01
         self.biases = np.random.rand(nodes) * 0.01
         self.delta_weights = np.random.rand(self.values.size, nodes)
         self.delta_biases = np.random.rand(nodes)

      values= np.dot(self.values.reshape(1, self.values.size),self.weights).flatten() + self.biases
      zvalues=values
      for i,value in enumerate(values):
         value=sigmoid(value)
         values[i]=value

      layer=Layer(values)
      self.next_layer=layer
      layer.zvalues=zvalues
      layer.previous_layer=self
      return layer
    
    def forward(self,image):
      layer=self
      layer.values=image
      while layer.next_layer!=None:
        zvalues=np.dot(layer.values.reshape(1, layer.values.size),layer.weights).flatten() +layer.biases
        layer.next_layer.zvalues=zvalues
        for i,value in enumerate(zvalues):
            zvalues[i]=sigmoid(value)
        layer.next_layer.values=zvalues
        layer=layer.next_layer

    def backward(self):
      ...
          
    
    def __repr__(self):
      return f"Layer(values={self.values})"

    

In [97]:
def costFunction(actual, expectation):
    cost=None
    for i, value in enumerate(actual):
        cost=cost+(value-expectation[i])**2
    return cost


sample_image=training_data[0]
first_layer=Layer(sample_image)
second_layer=first_layer.nextLayer(300)
third_layer=second_layer.nextLayer(100)
fourth_layer=third_layer.nextLayer(10)
first_layer.forward(training_data[0])

[0.60739601 0.59786503 0.61625052 0.60589736 0.60554283 0.60827028
 0.60266673 0.60564227 0.61168988 0.59324139]
[0.60739601 0.59786503 0.61625052 0.60589736 0.60554283 0.60827028
 0.60266673 0.60564227 0.61168988 0.59324139]
